# Practice New SQL Features Released by Snowflake in 2024
### Author: [Prasanna Rajagopal](https://www.linkedin.com/in/prasannarajagopal/)
### Snowflake SQL Functions in this Notebook:
- [GREATEST](https://docs.snowflake.com/en/sql-reference/functions/greatest)
- [LEAST](https://docs.snowflake.com/en/sql-reference/functions/least)
- [GREATEST_IGNORE_NULLS](https://docs.snowflake.com/en/sql-reference/functions/greatest_ignore_nulls)
- [LEAST_IGNORE_NULLS](https://docs.snowflake.com/en/sql-reference/functions/least_ignore_nulls)
- Higher-Order Functions
    - [FILTER](https://docs.snowflake.com/en/sql-reference/functions/filter)
    - [TRANSFORM](https://docs.snowflake.com/en/sql-reference/functions/transform)
    - [REDUCE](https://docs.snowflake.com/en/sql-reference/functions/reduce)
- [COLLATION](https://docs.snowflake.com/en/sql-reference/functions/collation)
- [SEARCH](https://docs.snowflake.com/en/user-guide/querying-with-search-functions)
#### Last Updated: Mar 2025. 


# Pre-requisities
# Loading Data for Executing Queries
### We will load the share repurchase data for various companies from a CSV file into a table in Snowflake.
#### 1. Download the CSV file from [Github here](https://github.com/rrprasan/Finance/blob/main/Snowflake/Notebooks/Miscellaneous_Topics/2024_SQL_New_Features/US_Company_Share_Repurchase.csv). 
#### 2. We will create a new table and load data into it from the CSV file. 
#### 3. Follow the [instructions in this page](https://docs.snowflake.com/en/user-guide/data-load-web-ui#create-a-new-table-using-sf-web-interface) to create and load data into a table named - US_COMPANY_SHARE_REPURCHASE_TBL
-   We will use the [Create Table From File](https://docs.snowflake.com/en/user-guide/data-load-web-ui#create-a-new-table-using-sf-web-interface) in Snowflake Snowsight to create the table and load data from the CSV. 
#### 4. Please ensure the table is named correctly. The queries use the following name:
- US_COMPANY_SHARE_REPURCHASE_TBL


### Test the US_COMPANY_SHARE_REPURCHASE_TBL with a SELECT statement.
#### The query should return a total of 12 rows. 
#### Each row represents the annual amount (in USD) spent on share repurchases by Microsoft, Apple, Alphabet, Meta, and Oracle. 
#### The data is from fiscal years 2013 to 2024. The data contains some NULL and 0 values.
#### This data set will be good to illustrate the difference between the GREATEST and the GREATEST_IGNORE_NULLS functions.

In [ ]:
SELECT * FROM US_COMPANY_SHARE_REPURCHASE_TBL;

## New [GREATEST_IGNORE_NULLS](https://docs.snowflake.com/en/sql-reference/functions/greatest_ignore_nulls) and [LEAST_IGNORE_NULLS](https://docs.snowflake.com/en/sql-reference/functions/least_ignore_nulls) functions.
### Release Date: [March 2024](https://docs.snowflake.com/en/release-notes/sql-improvements#sql-improvements-in-2024)

### Before we try the GREATEST_IGNORE_NULLS and LEAST_IGNORE_NULLS SQL functions, let's understand how the GREATEST and LEAST functions work.  

## We wish to compare the share repurchases of Microsoft, Apple, Alphabet, Meta, and Oracle. 
### We want to answer the following questions:
- #### Which company had the "greatest" (highest) annual spend on buy backs?
- #### Which company has the "least" (lowest) annual spend on share buy backs?
- #### This query example also features the support for trailing comma (after ORACLE column name in the SQL) in Snowflake SQL. 
    - Trailing comma support in Snowflake was introduced in [March 2024](https://docs.snowflake.com/en/release-notes/sql-improvements#sql-improvements-in-2024) 
- #### You will see that if a particular year has NULL for a company's share buy back, the GREATEST or LEAST both would return NULL (None).
    - The reasons behind this behavior is explained here in a markdown below using Snowflake Cortex AI.   

In [ ]:
SELECT
    FISCAL_YEAR, 
    GREATEST(MICROSOFT, APPLE, ALPHABET, META, ORACLE) GREATEST_ANNUAL_SHARE_BUY_BACK,
    LEAST(MICROSOFT, APPLE, ALPHABET, META, ORACLE) LEAST_ANNUAL_SHARE_BUY_BACK,
    MICROSOFT,
    APPLE,
    ALPHABET,
    META, 
    ORACLE,
FROM
    US_COMPANY_SHARE_REPURCHASE_TBL;

### Let's ask Snowflake Cortex AI why the GREATEST SQL function returns NULL when any one value is NULL.
#### We use the [COMPLETE](https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex) function, part of the Cortex AI LLM product suite, to answer this question.  
```
SELECT SNOWFLAKE.CORTEX.COMPLETE(
            'claude-3-5-sonnet', 
            'Why does the GREATEST SQL function return NULL, when any of its input values are NULL? 
             Why can\'t it ignore NULL values?');
```

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', 'Why does the GREATEST SQL function return NULL, when any of its input values are NULL? Why can\'t it ignore NULL values?');

### Answer Provide by Snowflake Cortex AI [COMPLETE](https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex) function using Anthropic's Claude Sonnet Model.  
The GREATEST function returns NULL when any input is NULL because of SQL's _**three-valued logic (TRUE, FALSE, NULL) and how NULL represents unknown values.**_ 

Here's the reasoning:

1. When comparing values where one is NULL, the result is UNKNOWN (NULL) because you can't determine if an unknown value is greater than or less than a known value.

Example:
```sql
-- We can't know if NULL is greater than 10 or not
SELECT GREATEST(10, 5, NULL, 8); -- Returns NULL
```

2. If the function ignored NULL values, it could lead to incorrect results in cases where the NULL might actually represent a value larger than all other inputs.

Consider this scenario:
```sql
-- If NULL represents 100, ignoring it would give wrong result
SELECT GREATEST(10, 5, NULL, 8); -- Currently returns NULL
-- If it ignored NULLs, it would return 10, which could be wrong if NULL was actually 100
```

If you want to ignore NULL values, you can use GREATEST_IGNORE_NULLS.

This behavior is consistent with SQL's NULL handling principles and helps prevent incorrect assumptions about unknown values.


## GREATEST_IGNORE_NULLS and LEAST_IGNORE_NULLS
### Now let's apply GREATEST_IGNORE_NULLS and LEAST_IGNORE_NULLS on the share repurchase Data to ignore the NULL values and return results.  
#### Release Date: March 2024.

In [ ]:
SELECT
    FISCAL_YEAR,
    GREATEST_IGNORE_NULLS(APPLE, META, ALPHABET, MICROSOFT, ORACLE) GREATEST_SHARE_REPURCHASE,
    LEAST_IGNORE_NULLS(APPLE, META, ALPHABET, MICROSOFT, ORACLE) LEAST_SHARE_REPURCHASE,
    APPLE,
    META,
    ALPHABET,
    MICROSOFT,
    ORACLE,
FROM
    US_COMPANY_SHARE_REPURCHASE_TBL;

## Key differences between MAX and GREATEST SQL functions:

**MAX Function:**
1. **Aggregate function** that operates on a column across multiple rows
2. Returns the highest value in a column
3. Can only work with one column at a time
4. Used in GROUP BY operations
5. Ignores NULL values

Example:
```sql
SELECT MAX(salary) FROM employees;
SELECT department, MAX(salary) 
FROM employees 
GROUP BY department;
```

**GREATEST Function:**
1. **Scalar function** that compares multiple values/expressions in a single row
2. Returns the highest value among the provided arguments
3. Can compare multiple values/columns **within the same row**
4. Returns NULL if any argument is NULL
5. Not all databases support GREATEST (e.g., older versions of SQL Server)

Example:
```sql
SELECT employee_id, 
       GREATEST(salary, bonus, commission) as highest_payment 
FROM employees;

## GREATEST and LEAST SQL functions supports all data types, including VARIANT.
### Here we are doing a [lexicographical comparison](https://en.wikipedia.org/wiki/Lexicographic_order) to the find the "greatest" word.

1. Lexicographical Comparison: SQL compares strings character by character from left to right.  It uses the alphabetical order to determine which character is "greater".

2. Comparing the First Characters:

    'CAT' starts with 'C'\
    'BAT' starts with 'B'\
    'DOG' starts with 'D'

3. Determining the Greatest Based on the First Character:

    In the alphabet, 'B' comes before 'C', and 'C' comes before 'D'.
    Therefore, 'D' is the "greatest" first character among 'B', 'C', and 'D'.

In [ ]:
SELECT GREATEST('CAT', 'BAT', 'DOG');

LEAST Returns 'BAT' from LEAST('CAT', 'BAT', 'DOG')

In [ ]:
SELECT LEAST('CAT','BAT', 'DOG');

## Finding the GREATEST Array

In [ ]:
SELECT GREATEST(
                [23, 10, 15],
                [20, 11, 16],
                [25,  1,  2]
               );

### If the first value in the array is the same, the GREATEST function compares the second value in the query.  

In [ ]:
SELECT GREATEST(
                [23, 10, 15],
                [23, 11, 16],
                [23,  1,  2]
               );

#### Using Anthropic's Claude Model with Snowflake's COMPLETE LLM SQL function to explain the query's result. 
```
SELECT SNOWFLAKE.CORTEX.COMPLETE(
                                    'claude-3-5-sonnet', 
                                    'You are a helpful tool who explains SQL queries and why the query 
                                     returns a certain answer. 
                                     Look at the <query> and the <result> and provide an explanation. 
                                     <query>SELECT GREATEST( [23, 10, 15], [20, 11, 16], [25, 1, 2] );</query> 
                                     <result>[25, 1, 2]</result>');
```

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', 'You are a helpful tool who explains SQL queries and why the query returns a certain answer. Look at the <query> and the <result> and provide an explanation. <query>SELECT GREATEST( [23, 10, 15], [20, 11, 16], [25, 1, 2] );</query> <result>[25, 1, 2]</result>');

## Answer provided by Snowflake Cortex AI using the COMPLETE function and Anthropic's Claude Sonnet model:

Let me explain this SQL query and its result:

The query uses the GREATEST() function, which compares multiple values or arrays and returns the greatest value among them. In this case, it's comparing three arrays:
1. [23, 10, 15]
2. [20, 11, 16]
3. [25, 1, 2]

When GREATEST() compares arrays, it uses lexicographical ordering, which means:
1. It first compares the first elements of each array (23 vs 20 vs 25)
2. If the first elements are equal, it moves to the second elements, and so on

In this case:
- First array starts with 23
- Second array starts with 20
- Third array starts with 25

Since 25 is the greatest first element, the entire array [25, 1, 2] is returned as the result, regardless of the other numbers in the array. The comparison stops at the first element because it found a clear "winner."

That's why the result is [25, 1, 2], even though this array contains smaller numbers (1 and 2) in the second and third positions.

## Applying the GREATEST function on a 3-Dimensional Array.

In [ ]:
SELECT GREATEST(
            [
             [23, 10, 15],
             [41, 198, 20],
             [8, 1005, 10908]
            ], 
            [[20, 11, 16],
             [678, 267, 2000],
             [345, 2, 678]
            ], 
            [
            [25,1, 2],
            [8, 5, 3],
            [1, 22, 5]
            ])

## [FILTER](https://docs.snowflake.com/en/sql-reference/functions/filter) Higher-Order Function
#### Release Date: May 2024

### Let's use Snowflake Cortex AI and the Anthropic's Claude Sonnet model to help us better understand the FILTER SQL function. 
```
SELECT SNOWFLAKE.CORTEX.COMPLETE(
                                    'claude-3-5-sonnet', 
                                    'Explain the FILTER Snowflake SQL function in detail. 
                                     Filters an array based on the logic in a lambda expression. 
                                     It falls under the category of working with Semi-structured and 
                                     structured data functions');
```

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', 'Explain the FILTER Snowflake SQL function in detail. Filters an array based on the logic in a lambda expression. It falls under the category of working with Semi-structured and structured data functions');

### Execute the SQL statement above to get an answer to the question about FILTER SQL function. You can also expand the next markdown cell to see the complete answer.  

## Answer Provided by Snowflake Cortex AI and Anthropic's Claude Sonnet Model. 
The FILTER function in Snowflake SQL is a powerful array function that allows you to filter elements from an array based on a specified condition using a lambda expression. **It's particularly useful when working with semi-structured data (arrays) and you need to selectively extract elements that meet certain criteria.**\
Here's a detailed explanation with examples:

Syntax:
```sql
FILTER(array, lambda_expression)
```

Key Points:
1. The function takes two parameters:
   - array: The input array to be filtered
   - lambda_expression: A condition that determines which elements to keep

2. Returns a new array containing only the elements that satisfy the condition

Examples:

1. Basic Filtering:
```sql
-- Filter numbers greater than 3
SELECT FILTER(ARRAY_CONSTRUCT(1, 2, 3, 4, 5), x -> x > 3);
-- Result: [4, 5]

-- Filter strings containing 'a'
SELECT FILTER(ARRAY_CONSTRUCT('apple', 'banana', 'orange'), x -> CONTAINS(x, 'a'));
-- Result: ['apple', 'banana', 'orange']
```

2. Working with JSON Data:
```sql
WITH sample_data AS (
  SELECT PARSE_JSON('
    {
      "items": [
        {"id": 1, "price": 10},
        {"id": 2, "price": 20},
        {"id": 3, "price": 30}
      ]
    }
  ') as json_data
)

SELECT FILTER(json_data:items, x -> x:price::number > 15)
FROM sample_data;
-- Result: [{"id": 2, "price": 20}, {"id": 3, "price": 30}]
```

3. Multiple Conditions:
```sql
-- Filter with multiple conditions
SELECT FILTER(
  ARRAY_CONSTRUCT(1, 2, 3, 4, 5, 6),
  x -> x > 2 AND x < 6
);
-- Result: [3, 4, 5]
```

4. Filtering Objects:
```sql
WITH products AS (
  SELECT ARRAY_CONSTRUCT(
    OBJECT_CONSTRUCT('name', 'Apple', 'price', 1.99, 'inStock', true),
    OBJECT_CONSTRUCT('name', 'Banana', 'price', 0.99, 'inStock', false),
    OBJECT_CONSTRUCT('name', 'Orange', 'price', 2.99, 'inStock', true)
  ) as items
)

SELECT FILTER(items, x -> x:price::float < 2.00 AND x:inStock::boolean = true)
FROM products;
-- Result: [{"name":"Apple","price":1.99,"inStock":true}]
```

5. Combining with Other Array Functions:
```sql
-- Filter and transform array elements
SELECT 
  ARRAY_AGG(x:name::string) 
FROM TABLE(FLATTEN(INPUT => 
  FILTER(
    ARRAY_CONSTRUCT(
      OBJECT_CONSTRUCT('name', 'Apple', 'price', 1.99),
      OBJECT_CONSTRUCT('name', 'Banana', 'price', 0.99),
      OBJECT_CONSTRUCT('name', 'Orange', 'price', 2.99)
    ),
    x -> x:price::float < 2.00
  )
));
-- Result: ['Banana']
```

Common Use Cases:

1. Filtering JSON Arrays:
```sql
-- Filter JSON array based on specific conditions
SELECT FILTER(
  PARSE_JSON('[
    {"id": 1, "status": "active"},
    {"id": 2, "status": "inactive"},
    {"id": 3, "status": "active"}
  ]'),
  x -> x:status::string = 'active'
);
```

2. Data Cleaning:
```sql
-- Remove null or empty values from array
SELECT FILTER(
  ARRAY_CONSTRUCT('a', null, 'b', '', 'c'),
  x -> x IS NOT NULL AND x != ''
);
```

3. Complex Filtering:
```sql
-- Filter based on multiple conditions and transformations
SELECT FILTER(
  ARRAY_CONSTRUCT(1, 2, 3, 4, 5, 6),
  x -> MOD(x, 2) = 0 AND POWER(x, 2) < 25
);
```

Best Practices:

1. Always handle NULL values appropriately:
```sql
-- Safe handling of NULL values
SELECT FILTER(
  array_col,
  x -> COALESCE(x:value::number, 0) > 0
)
FROM your_table;
```

2. Use type casting when working with JSON:
```sql
-- Explicit type casting
SELECT FILTER(
  json_array,
  x -> TRY_CAST(x:amount::string AS NUMBER) > 100
);
```

3. Combine with other functions for complex operations:
```sql
-- Combining FILTER with TRANSFORM
SELECT 
  TRANSFORM(
    FILTER(array_col, x -> x > 0),
    x -> POWER(x, 2)
  )
FROM your_table;
```

The FILTER function is particularly useful when:
- Working with JSON data
- Processing arrays of values
- Cleaning data
- Implementing complex business logic on array elements
- Handling semi-structured data

# Pre-requisites for Testing FILTER SQL Function
## We will apply the FILTER SQL function on the revenue data of various U.S. Companies. 
### This data is downloaded from the U.S. SEC.  
#### 1. Please download the file - [Company_Revenue_JSON.zip](https://github.com/rrprasan/Finance/blob/main/Snowflake/Notebooks/Miscellaneous_Topics/2024_SQL_New_Features/Company_Revenue_JSON.zip) - from Github.
#### 2. This zip file contains the revenue JSON files for the following companies:
    - Colgate-Palmolive Company - CL_revenues.json
    - Costco Wholsale - COST_revenues.json
    - Coca-Cola Co - KO_revenues.json
    - PepsiCo Inc., - PEP_revenues.json
    - Target Corporation - TGT_revenues.json
    - Walmart Inc. - WMT_revenues.json
#### 3. Unzip the files into your local drive.  
#### 4. Follow the [instructions in this page](https://docs.snowflake.com/en/user-guide/data-load-web-ui#create-a-new-table-using-sf-web-interface) to create and load data into a table named - US_COMPANY_REVENUE_TBL
- Please ensure the VARIANT column is named REVENUE_JSON
- The [instructions](https://docs.snowflake.com/en/user-guide/data-load-web-ui#create-a-new-table-using-sf-web-interface) point to creating and loading data into a table. 
- You can load six files into the "Drag and drop to Upload Files" box.
#### 5. Here's a sample of the revenue data as submitted to the SEC by Colgate-Palmolive Company (NYSE:[CL](https://www.cnbc.com/quotes/CL?qsearchterm=CL)): 
```json
{
 "cik":21665,
 "taxonomy":"us-gaap","tag":"Revenues",
 "label":"Revenues",
 "description":"Amount of revenue recognized from goods sold, services rendered, insurance premiums, or other 
  activities that constitute an earning process. Includes, but is not limited to, investment and interest income 
  before deduction of interest expense when recognized as a component of revenue, and sales and trading gain (loss).",
 "entityName":"COLGATE-PALMOLIVE COMPANY",
 "units":
    {
      "USD":
        [
            {
             "start":"2011-01-01",
             "end":"2011-12-31",
             "val":16734000000,
             "accn":"0001545547-14-000003",
             "fy":2013,
             "fp":"FY",
             "form":"10-K",
             "filed":"2014-02-20",
             "frame":"CY2011"
             },
            {
             "start":"2012-01-01",
             "end":"2012-06-30",
             "val":8467000000,
             "accn":"0001445305-13-001692",
             "fy":2013,
             "fp":"Q2",
             "form":"10-Q",
             "filed":"2013-07-25"
             }
        ]
    }
}
```
#### 6. We seek to answer this question: What are the annual revenues for each company? 
#### 7. The JSON file for each company contains both the annual and quarterly revenues. 
#### 8. We will use the FILTER function to return only the JSON dictionaries for each company's annual revenues.
#### 9. Annual revenues can be identified by the key "fp" and value "FY", for fiscal year.  

### Once the data is loaded into the table, we are ready to query the table.  
### We will apply the FILTER SQL function to only look at annual revenue data for each company. 
#### Please note, the SEC data may contain duplicate data for annual and quarterly revenues. This is due to the fact that revenue may be restated by the company or the data is repeated to allow for comparison between quarters or years.  

In [ ]:
SELECT
    CS.REVENUE_JSON:"entityName"::String COMPANY_NAME,
    OI.value:"start"::DATE FY_START_DATE,
    OI.value:"end"::DATE FY_END_DATE,
    OI.value:"val"::NUMBER FY_REVENUE,
FROM
    US_COMPANY_REVENUE_TBL CS,
    LATERAL FLATTEN(input => FILTER(REVENUE_JSON:"units":"USD",a -> a:fp = 'FY')) oi
ORDER BY 
    COMPANY_NAME, FY_END_DATE DESC;

### Let's explain this query that uses the FILTER SQL function.
#### We use the Snowflake Cortex [COMPLETE](https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex) function with the Anthropic Claude model to explain the use of the FILTER sql function in a query.
* You can execute the COMPLETE function below to get a detailed explanation about the query. 

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', 'Explain this query is detail: <query>SELECT
    CS.REVENUE_JSON:"entityName"::String COMPANY_NAME,
    OI.value:"start"::DATE FY_START_DATE,
    OI.value:"end"::DATE FY_END_DATE,
    OI.value:"val"::NUMBER FY_REVENUE,
FROM
    US_COMPANY_REVENUE_TBL CS,
    LATERAL FLATTEN(input => FILTER(REVENUE_JSON:\"units\":\"USD\",a -> a:fp = \'FY\')) oi
ORDER BY 
    COMPANY_NAME, FY_END_DATE DESC</query>');

### The FILTER query explained by Snowflake Cortex COMPLETE function using Anthropic's Claude Sonnet LLM.
Let's break down this SQL query step by step:

1. FROM Clause:
```sql
FROM US_COMPANY_REVENUE_TBL CS,
LATERAL FLATTEN(input => FILTER(REVENUE_JSON:"units":"USD",a -> a:fp = 'FY')) oi
```
- Starts with base table `US_COMPANY_REVENUE_TBL` aliased as `CS`
- Uses LATERAL FLATTEN to **unnest JSON data**
- FILTER function is used to filter JSON elements where `fp = 'FY'`
- The filtered results are flattened (unnested) and aliased as `oi`

2. SELECT Clause:
```sql
SELECT
    CS.REVENUE_JSON:"entityName"::String COMPANY_NAME,
    OI.value:"start"::DATE FY_START_DATE,
    OI.value:"end"::DATE FY_END_DATE,
    OI.value:"val"::NUMBER FY_REVENUE,
```
- Extracts `entityName` from REVENUE_JSON and casts it to String
- Gets `start` date from the flattened data and casts to DATE
- Gets `end` date from the flattened data and casts to DATE
- Gets `val` (revenue value) from the flattened data and casts to NUMBER

3. ORDER BY Clause:
```sql
ORDER BY 
    COMPANY_NAME, FY_END_DATE DESC
```
- Sorts results first by COMPANY_NAME (ascending)
- Then by FY_END_DATE in descending order (most recent dates first)

The query appears to be working with JSON data containing company revenue information:
- It specifically looks at USD currency units
- Filters for fiscal year ('FY') records
- Extracts company names and their fiscal year revenue details
- Returns the data in a structured format, sorted by company and date

Expected output would include:
- Company names
- Fiscal year start dates
- Fiscal year end dates
- Revenue values
Ordered by company name and most recent fiscal years first.

## [TRANSFORM](https://docs.snowflake.com/en/sql-reference/functions/transform) Higher-order Function
Release Date: [May 2024](https://docs.snowflake.com/en/release-notes/sql-improvements#sql-improvements-in-2024)

### Transforms an array based on the logic in a lambda expression.

#### We will transfrom the revenue data as submitted to the SEC by Colgate-Palmolive Company (NYSE:[CL](https://www.cnbc.com/quotes/CL?qsearchterm=CL)) for this example: 
```json
{
 "cik":21665,
 "taxonomy":"us-gaap",
 "tag":"Revenues",
 "label":"Revenues",
 "description":"Amount of revenue recognized from goods sold, services rendered, insurance premiums, or other 
  activities that constitute an earning process. Includes, but is not limited to, investment and interest income 
  before deduction of interest expense when recognized as a component of revenue, and sales and trading gain (loss).",
 "entityName":"COLGATE-PALMOLIVE COMPANY",
 "units":
    {
      "USD":
        [
            {
             "start":"2011-01-01",
             "end":"2011-12-31",
             "val":16734000000,
             "accn":"0001545547-14-000003",
             "fy":2013,
             "fp":"FY",
             "form":"10-K",
             "filed":"2014-02-20",
             "frame":"CY2011"
             },
            {
             "start":"2012-01-01",
             "end":"2012-06-30",
             "val":8467000000,
             "accn":"0001445305-13-001692",
             "fy":2013,
             "fp":"Q2",
             "form":"10-Q",
             "filed":"2013-07-25"
             }
        ]
    }
}
```
#### We aim to use the "TRANSFORM" SQL function to calculate the number of days in the fiscal period by calculating the difference between the "end" and "start" date elements in the JSON. If the days are approximately 90, the revenue is for the quarter; if they are approximately 180 days, the revenue is for six months, and so on. 
#### There are other ways in SQL to accomplish this goal, but, for this example, we will use the TRANSFORM function.  
#### You can do much more to manipulate data in arrays using the TRANFORM function.  

In [ ]:
SELECT * FROM US_COMPANY_REVENUE_TBL CS;

### Use the TRANSFORM SQL function to retrieve an array of CONCATENATED "start" and "end" date.
### Use the TRANSFORM SQL function to calculate the difference in days between the "end" and the "start" dates to view the number of days in the fiscal period.  
### We are **not un-nesting the JSON** using LATERAL FLATTEN. 

In [ ]:
SELECT
    TRANSFORM(REVENUE_JSON:"units":"USD", a -> a:"start" || ' ' || a:"end") FP_DATE,
    TRANSFORM(REVENUE_JSON:"units":"USD", a -> a:"end"::DATE - a:"start"::DATE) DAYS_IN_FISCAL_PERIOD,
    CS.REVENUE_JSON:"entityName"::VARCHAR COMPANY_NAME,
FROM
    US_COMPANY_REVENUE_TBL CS
ORDER BY COMPANY_NAME;

### The Snowflake Cortex AI COMPLETE function to explain the query.  

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', 'Explain this query is detail: <query>SELECT
    TRANSFORM(REVENUE_JSON:"units":"USD", a -> a:"start" || \' \' || a:"end") FP_DATE,
    TRANSFORM(REVENUE_JSON:"units":"USD", a -> a:"end"::DATE - a:"start"::DATE) DAYS_IN_FISCAL_PERIOD,
    CS.REVENUE_JSON:"entityName"::VARCHAR COMPANY_NAME,
FROM
    US_COMPANY_REVENUE_TBL CS
ORDER BY COMPANY_NAME</query>');

### Query Explained by Snowflake Cortex COMPLETE SQL LLM function using Anthropic Claude LLM model. 

Let's break down this SQL query and explain each part in detail:

1. The FROM Clause:
```sql
FROM US_COMPANY_REVENUE_TBL CS
```
- This query is selecting from a table named `US_COMPANY_REVENUE_TBL`
- The table is given an alias 'CS'

2. The SELECT Clause contains three main transformations:

a. First TRANSFORM:
```sql
TRANSFORM(REVENUE_JSON:"units":"USD", a -> a:"start" || ' ' || a:"end") FP_DATE
```
- This is accessing a JSON structure in column REVENUE_JSON, navigating to "units"->"USD"
- For each element in this array, it's concatenating the "start" and "end" values with a space between them
- The result is aliased as FP_DATE
- The syntax `a -> a:"start"` is a lambda function where 'a' represents each element in the array

b. Second TRANSFORM:
```sql
TRANSFORM(REVENUE_JSON:"units":"USD", a -> a:"end"::DATE - a:"start"::DATE) DAYS_IN_FISCAL_PERIOD
```
- Similar JSON path access as above
- Converts both "start" and "end" values to DATE type
- Calculates the difference between end and start dates
- Results in the number of days in each fiscal period
- Aliased as DAYS_IN_FISCAL_PERIOD

c. Third column:
```sql
CS.REVENUE_JSON:"entityName"::VARCHAR COMPANY_NAME
```
- Accesses the "entityName" field from the REVENUE_JSON column
- Casts the result to VARCHAR type
- Aliased as COMPANY_NAME

3. The ORDER BY Clause:
```sql
ORDER BY COMPANY_NAME
```
- Sorts the final results alphabetically by COMPANY_NAME

Expected Output:
- The query will return a result set with three columns:
  1. FP_DATE: Combined start and end dates as a string
  2. DAYS_IN_FISCAL_PERIOD: Number of days between start and end dates
  3. COMPANY_NAME: Name of the company

The data appears to be financial/revenue data stored in a JSON format, and this query is extracting and transforming specific elements from that JSON structure while calculating date ranges for fiscal periods.

### TRANSFORM with LATERAL FLATTEN

In [ ]:
SELECT
    OI.VALUE:"start"::DATE FP_START_DATE,
    OI.VALUE:"end"::DATE FP_END_DATE,
    TRANSFORM(VALUE, a -> a:"end"::DATE - a:"start"::DATE)[0] DAYS_IN_FISCAL_PERIOD,
    CS.REVENUE_JSON:"entityName"::VARCHAR COMPANY_NAME,
    TO_VARCHAR(VALUE:"val"::NUMBER, '$999,999,999,999')  FP_COMPANY_REVENUE,
FROM
    US_COMPANY_REVENUE_TBL CS,
    LATERAL FLATTEN(input => REVENUE_JSON:"units":"USD") oi
ORDER BY COMPANY_NAME, FP_START_DATE DESC, DAYS_IN_FISCAL_PERIOD DESC;

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', 'Explain this query is detail: <query>SELECT
    OI.VALUE:\"start\"::DATE FP_START_DATE,
    OI.VALUE:\"end\"::DATE FP_END_DATE,
    TRANSFORM(VALUE, a -> a:\"end\"::DATE - a:\"start\"::DATE)[0] DAYS_IN_FISCAL_PERIOD,
    CS.REVENUE_JSON:\"entityName\"::VARCHAR COMPANY_NAME,
    TO_VARCHAR(VALUE:\"val\"::NUMBER, \'$999,999,999,999\')  FP_COMPANY_REVENUE,
FROM
    US_COMPANY_REVENUE_TBL CS,
    LATERAL FLATTEN(input => REVENUE_JSON:\"units\":\"USD\") oi
ORDER BY COMPANY_NAME, FP_START_DATE DESC, DAYS_IN_FISCAL_PERIOD DESC;</query>');

Let me break down this SQL query in detail:

1. FROM Clause:
- The query is selecting from a table named `US_COMPANY_REVENUE_TBL` (aliased as CS)
- It uses LATERAL FLATTEN to unnest a JSON array found in the REVENUE_JSON column, specifically the nested path "units":"USD"

2. SELECT Clause:
```sql
OI.VALUE:"start"::DATE FP_START_DATE
```
- Extracts the "start" field from the flattened JSON and casts it to a DATE
- Aliases it as FP_START_DATE

```sql
OI.VALUE:"end"::DATE FP_END_DATE
```
- Extracts the "end" field from the flattened JSON and casts it to a DATE
- Aliases it as FP_END_DATE

```sql
TRANSFORM(VALUE, a -> a:"end"::DATE - a:"start"::DATE)[0] DAYS_IN_FISCAL_PERIOD
```
- Uses TRANSFORM function to calculate the difference between end and start dates
- Returns the number of days in the fiscal period
- [0] indicates it's taking the first element of the transformed array

```sql
CS.REVENUE_JSON:"entityName"::VARCHAR COMPANY_NAME
```
- Extracts the "entityName" field from REVENUE_JSON and casts it to VARCHAR
- Aliases it as COMPANY_NAME

```sql
TO_VARCHAR(VALUE:"val"::NUMBER, '$999,999,999,999') FP_COMPANY_REVENUE
```
- Extracts the "val" field from the flattened JSON, casts it to NUMBER
- Formats it as a string with currency formatting
- Aliases it as FP_COMPANY_REVENUE

3. ORDER BY Clause:
```sql
ORDER BY COMPANY_NAME, FP_START_DATE DESC, DAYS_IN_FISCAL_PERIOD DESC
```
- Orders the results first by COMPANY_NAME (ascending)
- Then by FP_START_DATE in descending order
- Finally by DAYS_IN_FISCAL_PERIOD in descending order

This query appears to be analyzing financial data, specifically revenue information for US companies, breaking it down by fiscal periods and formatting it in a readable way with proper currency formatting.

## [REDUCE](https://docs.snowflake.com/en/sql-reference/functions/reduce) Higher-Order Function
Reduces an array to a single value based on the logic in a lambda expression.

The REDUCE function takes an array, an initial accumulator value, and a lambda function. It applies the lambda function to each element of the array, updating the accumulator with each result. After processing all elements, REDUCE returns the final accumulator value.

Release Date: [October 2024](https://docs.snowflake.com/en/release-notes/sql-improvements#sql-improvements-in-2024)

### We want to find all the fiscal end periods represented in each company's revenue JSON document. 
### We use the REDUCE higher-order function to accumulate all the fiscal end periods represented in the document.
### This SQL helps us understand the data represented in the JSON. It can also show any duplicates. 

In [ ]:
SELECT 
    REVENUE_JSON:"entityName" COMPANY_NAME, 
    REDUCE(REVENUE_JSON:"units":"USD",[], (arg1, arg2) -> ARRAY_APPEND(arg1, arg2:"end")) FISCAL_PERIODS
FROM 
    US_COMPANY_REVENUE_TBL;

## Explain the Query Using Snowflake Cortex AI

In [ ]:
SELECT SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet', 'Explain this query is detail: <query>SELECT 
    REVENUE_JSON:"entityName" COMPANY_NAME, 
    REDUCE(REVENUE_JSON:"units":"USD",[], (arg1, arg2) -> ARRAY_APPEND(arg1, arg2:"end")) FISCAL_PERIODS
FROM 
    US_COMPANY_REVENUE_TBL;');

### REDUCE SQL Query Explained using Snowflake Cortex AI COMPLETE function using the Anthropic Claude LLM model:

Let's break down this SQL query that appears to be working with JSON data:

1. FROM Clause:
- The query is selecting from a table named `US_COMPANY_REVENUE_TBL`

2. SELECT Clause has two main parts:

a) First Column: `REVENUE_JSON:"entityName" COMPANY_NAME`
- This extracts the "entityName" field from a JSON column named REVENUE_JSON
- The result is aliased as COMPANY_NAME

b) Second Column: `REDUCE(REVENUE_JSON:"units":"USD",[], (arg1, arg2) -> ARRAY_APPEND(arg1, arg2:"end")) FISCAL_PERIODS`
- This is using the REDUCE function to process JSON data
- It's working with the path REVENUE_JSON:"units":"USD"
- Starting with an empty array []
- For each element, it's applying a function that:
  * Takes two arguments (arg1, arg2)
  * Appends the "end" value from arg2 to arg1
- The result is aliased as FISCAL_PERIODS

The query essentially:
1. Takes JSON data about company revenues
2. Extracts the company name
3. Creates an array of fiscal period end dates from USD unit data
4. Returns both pieces of information for each company

Example of possible output:
```
COMPANY_NAME    FISCAL_PERIODS
ABC Corp       ["2021-12-31", "2022-12-31"]
XYZ Inc        ["2021-03-31", "2022-03-31"]
```

This query appears to be using Snowflake's JSON handling capabilities, given the syntax for JSON path navigation (using ":") and the REDUCE function.

# What's New in Collation?
- ## Support for the upper, lower, and trim [collations](https://docs.snowflake.com/en/sql-reference/collation).

# Understanding Collation
- Text strings in Snowflake are stored using the UTF-8 character set and, by default, strings are compared according to the Unicode codes that represent the characters in the string.
- However, comparing strings based on their UTF-8 character representations might not provide the desired or expected behavior. For example:
- If special characters in a given language do not sort according to that language’s ordering standards, then sorting might return unexpected results.
- You might want the strings to be ordered by other rules, such as ignoring whether the characters are uppercase or lowercase.

Collation allows you to explicitly specify the rules to use for comparing strings, based on:

- Different locales (that is, different character sets for different languages).

- Case-sensitivity (that is, whether to use case-sensitive or case-insensitive string comparisons without explicitly calling the UPPER or LOWER functions to convert the strings).

- Accent-sensitivity (for example, whether Z, Ź, and Ż are considered the same letter or different letters).

- Punctuation-sensitivity (that is, whether comparisons use only letters or include all characters). For example, if a comparison is punctuation-insensitive, then A-B-C and ABC are treated as equivalent.

- Additional options, such as preferences for sorting based on the first letter in a string and trimming of leading and/or trailing blank spaces.

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE collation_demo (
  uncollated_phrase VARCHAR, 
  utf8_phrase VARCHAR COLLATE 'utf8',
  english_phrase VARCHAR COLLATE 'en',
  spanish_phrase VARCHAR COLLATE 'es');

In [ ]:
INSERT INTO collation_demo (
      uncollated_phrase, 
      utf8_phrase, 
      english_phrase, 
      spanish_phrase) 
   VALUES (
     'pinata', 
     'pinata', 
     'pinata', 
     'piñata');

In [ ]:
SELECT * FROM collation_demo;

### This query that compares to pinata, returns true for uncollated, utf8, and english phrases.  
### The Spanish Phrase does not match the "pinata"

In [ ]:
SELECT uncollated_phrase = 'pinata', 
       utf8_phrase = 'pinata', 
       english_phrase = 'pinata', 
       spanish_phrase = 'pinata'
FROM collation_demo;

### The following query does not find a match because the Spanish character ñ does not match n:

In [ ]:
SELECT * FROM collation_demo WHERE spanish_phrase = uncollated_phrase;

## Collation Demo # 2

In [ ]:
CREATE OR REPLACE TRANSIENT TABLE collation_demo1 (
  uncollated_phrase VARCHAR, 
  utf8_phrase VARCHAR COLLATE 'utf8',
  english_phrase VARCHAR COLLATE 'en-ai',
  spanish_phrase VARCHAR COLLATE 'es-ai');

In [ ]:
INSERT INTO collation_demo1 (
    uncollated_phrase, 
    utf8_phrase, 
    english_phrase, 
    spanish_phrase) 
  VALUES (
    'piñata', 
    'piñata', 
    'piñata', 
    'piñata');

## Changing collation doesn’t force related, but unequal, characters (for example, ñ and n) to be treated as equal:

- ### Only the English phrase returns True for the following reasons:

- ### Uncollated comparisons don’t ignore accents.

- ### utf8 collation comparisons don’t ignore accents.

The en-ai and es-ai collation comparisons ignore accents, but in Spanish, ñ is treated as an individual character rather than an accented n.
(Source: Snowflake Docs on [COLLATION](https://docs.snowflake.com/en/sql-reference/collation#collation-examples))

In [ ]:
SELECT uncollated_phrase = 'pinata', 
       utf8_phrase = 'pinata', 
       english_phrase = 'pinata', 
       spanish_phrase = 'pinata'
  FROM collation_demo1;

## Searching Text in Snowflake Using [SEARCH](https://docs.snowflake.com/en/sql-reference/functions/search) function
- You can use search functions to find character data (text) in specified columns from one or more tables, including fields in VARIANT, OBJECT, and ARRAY columns. 
- This function searches the text in specified columns or strings based on a list of given search terms. 
- The function returns TRUE if the text matches the specified search terms based on the search semantics.
- In most cases, you call the SEARCH function by specifying it in the SELECT list or the WHERE clause of a SELECT statement. 
- If the function is used as a WHERE clause filter, the query returns rows when the function returns TRUE.
- Source: [Snowflake Documentation](https://docs.snowflake.com/en/user-guide/querying-with-search-functions)

### We will use the US_COMPANY_REVENUE_TBL created earlier in the Notebook for learning the Search function.  
### This table contains ths RAW JSON from the SEC filings of various companies.  
### This table contains one column called REVENUE_JSON that contains the JSON document.  
### We will use SEARCH in the WHERE clause to filter the JSON documents based on the company name.
### For example, SEARCH for Costco, Colgate, etc.  

### Execute a SELECT statement on the US_COMPANY_REVENUE_TBL to see the JSON documents stored in the VARIANT column.  

In [ ]:
SELECT * FROM US_COMPANY_REVENUE_TBL;

## Filter the rows in the table by searching for Costco or Colgate. 

In [ ]:
SELECT * FROM US_COMPANY_REVENUE_TBL WHERE SEARCH((*), 'Costco');

## Search for Colgate

In [ ]:
SELECT * FROM US_COMPANY_REVENUE_TBL WHERE SEARCH((*), 'Colgate');

## Using the SEARCH function in the SELECT list.  
## The function returns True where the text is found.  

In [ ]:
SELECT SEARCH((*), 'Colgate'), REVENUE_JSON FROM US_COMPANY_REVENUE_TBL;